In [1]:
# import necessary packages
import os, sys
import glob
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from ipywidgets import FloatProgress
from PIL import Image
from scipy import ndimage
from importlib import reload

sys.path.append('../')

import utils.Preprocessing as Preprocessing

In [2]:
# options
image_path = "../data/train/train_images/"
mask_path = "../data/train/train_masks_preproc/"
image_save_path = "../data/train/train_images_by_pod/"
mask_save_path = "../data/train/train_masks_by_pod/"

plot = False

# load image data
image_names = glob.glob(mask_path + "*.png")
image_names = [os.path.basename(x) for x in image_names]

In [3]:
# split train masks
reload(Preprocessing)
Preprocessing.split_image(image_names=image_names,
                          image_path=image_path,
                          image_save_path=image_save_path,
                          mask_path=mask_path,
                          mask_save_path=mask_save_path,
                          plot=False)


Processing image 16/17:  89.ISU386_B.B.ISU Wild.W.png


  0%|          | 0/10 [00:00<?, ?it/s]

Image Splitting Complete!
